In [1]:
import numpy as np 
import pandas as pd 
import tensorflow as tf 
import keras
from keras.layers import Dense, LSTM, SimpleRNN, GRU, Input
from keras.models import Model, Sequential  

df_train = pd.read_csv("./storage/Kaggle_M5_Luke/sales_train_validation.csv")
df_sell = pd.read_csv("./storage/Kaggle_M5_Luke/sell_prices.csv")
df_calendar = pd.read_csv("./storage/Kaggle_M5_Luke/calendar.csv")
df_ss = pd.read_csv("./storage/Kaggle_M5_Luke/sample_submission_m5.csv")


Using TensorFlow backend.


In [2]:
days = range(1,1913+1) 
time_series_cols = [] 
for i in days: 
    time_series_cols.append('d_'+str(i)) 
time_series_cols[:10]

['d_1', 'd_2', 'd_3', 'd_4', 'd_5', 'd_6', 'd_7', 'd_8', 'd_9', 'd_10']

In [3]:
time_series_data = df_train[time_series_cols] 
time_series_data

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,0,0,0,0,0,0,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,0,0,0,0,0,0,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,0,0,0,0,0,0,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,0,0,2,2,0,3,1,4,1,0,...,2,0,0,0,0,0,1,0,0,1
30486,0,0,0,0,0,5,0,1,1,3,...,0,0,0,0,0,0,0,0,1,0
30487,0,6,0,2,2,4,1,8,5,2,...,2,1,0,2,0,1,0,0,1,0
30488,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,3,1,3


In [4]:
time_series_data = np.asarray(time_series_data) 
print(time_series_data.shape)

(30490, 1913)


In [5]:
def build_model(time_steps,features): 
    model = Sequential() 
    model.add(LSTM(8,input_shape=(time_steps,features),activation='relu')) 
    model.add(Dense(30490)) 
    return model 
#model = build_model(7,30490)
#model.summary()

In [7]:
# let's now code for the general case: for all the 28 days 
def split_xy2(dataset, time_steps):
    x,y = list(), list() 
    for i in range(0,dataset.shape[1]-time_steps): 
        x.append(dataset[:,i:i+time_steps]) 
        y.append(dataset[:,i+time_steps]) 
    return np.array(x), np.array(y) 

predictions = [] 

for i in range(1,29): 
    print("day {}".format(i)) 
    dataset = time_series_data[:,::i] 
    x_train, y_train = split_xy2(dataset,7) 
    x_pred = dataset[:,dataset.shape[1]-7:] 
    x_train = x_train.reshape(x_train.shape[0],7,30490) 
    x_pred = x_pred.reshape(1,7,30490) 
    model = build_model(7,30490) 
    model.compile(loss='mse',optimizer='adam',metrics=['mse'])
    model.fit(x_train,y_train,epochs=100) 
    pred = model.predict(x_pred) 
    predictions.append(pred) 
    


day 1
Epoch 1/100
1906/1906 [==============================] - 4s 2ms/step - loss: 16.2641 - mse: 16.2641
Epoch 2/100
1906/1906 [==============================] - 3s 2ms/step - loss: 16.0706 - mse: 16.0706
Epoch 3/100
1906/1906 [==============================] - 3s 2ms/step - loss: 15.9549 - mse: 15.9549
Epoch 4/100
1906/1906 [==============================] - 3s 2ms/step - loss: 15.8445 - mse: 15.8445
Epoch 5/100
1906/1906 [==============================] - 3s 2ms/step - loss: 15.7393 - mse: 15.7393
Epoch 6/100
1906/1906 [==============================] - 3s 2ms/step - loss: 15.6387 - mse: 15.6387
Epoch 7/100
1906/1906 [==============================] - 3s 2ms/step - loss: 15.5423 - mse: 15.5423
Epoch 8/100
1906/1906 [==============================] - 3s 2ms/step - loss: 15.4498 - mse: 15.4498A: 0s - loss: 15.4291 - 
Epoch 9/100
1906/1906 [==============================] - 3s 2ms/step - loss: 15.3607 - mse: 15.3607
Epoch 10/100
1906/1906 [==============================] - 3s 2ms/step 

In [24]:
predictions = np.asarray(predictions) 
print(predictions.shape)

(28, 1, 30490)


In [44]:
# flatten each of predictions[i] 
pred_flattened = [] 
for i in range(28): 
    pred_flattened.append(predictions[i].flatten()) 

In [45]:
# checking if shape is flattened 
pred_flattened[0]

array([0.31285957, 0.2598569 , 0.15093106, ..., 0.8908798 , 0.36867678,
       0.6967665 ], dtype=float32)

In [52]:
cols = [] 
for i in range(1,29): 
    cols.append("F"+str(i)) 
df = pd.DataFrame(pred_flattened)   
df = df.T # transpose 
df.columns = cols # rename column 
df 

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,0.312860,0.634998,0.296127,0.295624,0.360747,0.271643,0.484471,0.510944,0.305839,0.269604,...,0.421772,0.188584,0.236893,0.183583,0.518191,0.280969,0.548796,0.195214,0.783453,0.897807
1,0.259857,0.207777,0.122160,0.259181,0.264924,0.195073,0.077438,0.303518,0.281660,0.244295,...,0.231701,0.157559,0.240345,0.236178,0.485402,0.102550,0.165629,0.189551,0.319089,0.181170
2,0.150931,0.244005,0.219131,0.113483,0.181141,0.222292,0.027066,0.199564,0.194427,0.128669,...,0.131690,0.101426,0.202999,0.109908,0.350022,0.217465,0.166407,0.103997,0.693319,1.163689
3,1.724048,2.030976,1.807238,1.160853,0.971213,1.668687,2.492478,2.282120,0.616268,0.534958,...,2.243176,0.274885,0.288460,0.284480,1.950832,2.274380,1.478957,0.270757,3.415339,2.202817
4,0.971688,0.998900,0.747155,0.842916,0.829498,0.817892,1.701581,1.244291,0.566359,0.491422,...,1.309867,0.272758,0.283394,0.272409,1.138067,0.960227,0.941674,0.250163,1.451205,1.090119
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,0.532661,0.459930,0.294223,0.498654,0.473160,0.273490,0.402173,0.478580,0.377179,0.374400,...,0.496744,0.217778,0.215531,0.225859,0.348972,0.499187,0.115225,0.146893,0.487744,0.358081
30486,0.374481,0.116509,0.161508,0.381916,0.344509,0.296275,0.177988,0.266769,0.303043,0.267107,...,0.292811,0.161571,0.202195,0.215061,0.462614,0.373147,-0.072382,0.203863,0.008952,-0.033648
30487,0.890880,0.823331,0.439398,0.758415,0.661789,0.764113,1.109185,0.512946,0.519262,0.451212,...,0.847497,0.266927,0.269321,0.262192,0.886102,0.495878,0.773174,0.229846,1.151863,0.731443
30488,0.368677,0.609984,0.449790,0.414389,0.384841,0.371806,0.392959,0.710727,0.333464,0.345648,...,0.340482,0.237474,0.224510,0.197889,0.451420,0.153610,0.761950,0.178094,1.061167,1.345607


In [53]:
validation_ids = df_train['id'].values
evaluation_ids = [i.replace('validation', 'evaluation') for i in validation_ids]
ids = np.concatenate([validation_ids, evaluation_ids])       
ids

array(['HOBBIES_1_001_CA_1_validation', 'HOBBIES_1_002_CA_1_validation',
       'HOBBIES_1_003_CA_1_validation', ...,
       'FOODS_3_825_WI_3_evaluation', 'FOODS_3_826_WI_3_evaluation',
       'FOODS_3_827_WI_3_evaluation'], dtype=object)

In [54]:
sub_df = pd.DataFrame(ids, columns=['id']) 
df = pd.concat([df]*2).reset_index(drop=True) 
sub_df = pd.concat([sub_df,df],axis=1) 
sub_df.to_csv('LSTM_base2.csv',index=False) 

In [55]:
sub_df 

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.312860,0.634998,0.296127,0.295624,0.360747,0.271643,0.484471,0.510944,0.305839,...,0.421772,0.188584,0.236893,0.183583,0.518191,0.280969,0.548796,0.195214,0.783453,0.897807
1,HOBBIES_1_002_CA_1_validation,0.259857,0.207777,0.122160,0.259181,0.264924,0.195073,0.077438,0.303518,0.281660,...,0.231701,0.157559,0.240345,0.236178,0.485402,0.102550,0.165629,0.189551,0.319089,0.181170
2,HOBBIES_1_003_CA_1_validation,0.150931,0.244005,0.219131,0.113483,0.181141,0.222292,0.027066,0.199564,0.194427,...,0.131690,0.101426,0.202999,0.109908,0.350022,0.217465,0.166407,0.103997,0.693319,1.163689
3,HOBBIES_1_004_CA_1_validation,1.724048,2.030976,1.807238,1.160853,0.971213,1.668687,2.492478,2.282120,0.616268,...,2.243176,0.274885,0.288460,0.284480,1.950832,2.274380,1.478957,0.270757,3.415339,2.202817
4,HOBBIES_1_005_CA_1_validation,0.971688,0.998900,0.747155,0.842916,0.829498,0.817892,1.701581,1.244291,0.566359,...,1.309867,0.272758,0.283394,0.272409,1.138067,0.960227,0.941674,0.250163,1.451205,1.090119
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,FOODS_3_823_WI_3_evaluation,0.532661,0.459930,0.294223,0.498654,0.473160,0.273490,0.402173,0.478580,0.377179,...,0.496744,0.217778,0.215531,0.225859,0.348972,0.499187,0.115225,0.146893,0.487744,0.358081
60976,FOODS_3_824_WI_3_evaluation,0.374481,0.116509,0.161508,0.381916,0.344509,0.296275,0.177988,0.266769,0.303043,...,0.292811,0.161571,0.202195,0.215061,0.462614,0.373147,-0.072382,0.203863,0.008952,-0.033648
60977,FOODS_3_825_WI_3_evaluation,0.890880,0.823331,0.439398,0.758415,0.661789,0.764113,1.109185,0.512946,0.519262,...,0.847497,0.266927,0.269321,0.262192,0.886102,0.495878,0.773174,0.229846,1.151863,0.731443
60978,FOODS_3_826_WI_3_evaluation,0.368677,0.609984,0.449790,0.414389,0.384841,0.371806,0.392959,0.710727,0.333464,...,0.340482,0.237474,0.224510,0.197889,0.451420,0.153610,0.761950,0.178094,1.061167,1.345607


In [16]:
#model.compile(loss='mse',optimizer='adam',metrics=['mse'])
#model_checkpoint = keras.callbacks.ModelCheckpoint('./storage/Kaggle_M5_Luke/model_base2/model{epoch:02d}.h5',period=50)
#model.fit(x_train,y_train,epochs=1000,callbacks=[model_checkpoint],validation_data=(x_val,y_val))  
